## Загрузим нужные библиотеки

Идеи:
~~1) Переменная - количество авторов. ~~

2) OneHotEncoder - авторов в число (автор чувствует статью)

~~3) Отдельно выделить без авторов?~~

4) Определить популярность слов в теге, используя гугл трендс за период --> Интересно, но сложно

5) Определять выходные (например). => Поиграться с датой (мб часы утренние/вечерние)

6) Заменить категорию, сессию, автора (и/или) на среднее значение по frp (например) (Если много, то взять из этого максимум)

7) Аналогично с тегами (Узнать че за теги (изи) + популярность)

8) Распарсить сайт для поиска авторов и посчитать их среднюю популярность тип. (За 10 статей, например)

9) Получить длину слов в статье (Влияет как минимум на процент читателей полность прочитавших статью)

~~10) Предсказать на основе View и Depth frp~~

11) Сгруппировать по сессии??

12) Добавить флаг на наличие в статье картинки

13) Курс доллара в этот момент??

In [1]:
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV


In [23]:
import numpy as np
import xgboost as xg
from sklearn.model_selection import cross_validate

Выполним загрузу датсета

In [3]:
df_train = pd.read_csv("content/train_dataset_train.csv", index_col= 0)
df_test = pd.read_csv("content/test_dataset_test.csv", index_col= 0)
sample = pd.read_csv("content/sample_solution.csv", index_col= 0)

In [4]:
df_train.head(3)

,title,publish_date,session,authors,ctr,category,tags,views,depth,full_reads_percent
document_id,,,,,,,,,,
624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ,Европейский банк развития приостановил доступ ...,2022-04-04 10:29:44,IDE7mtH4RBqGn-8MXfGffQ,[],1.580,5409f11ce063da9c8b588a18,"['55928d339a794751dc8303d6', '542d1e28cbb20f86...",20460,1.134,35.850
620f6b899a7947701cf489e1KtVJsteHStO5oditt3Uvzw,Кремль назвал регулярным процессом учебные зап...,2022-02-18 10:00:39,KtVJsteHStO5oditt3Uvzw,"['54244e01cbb20f03076b236d','5878a2ec9a7947e53...",1.853,5409f11ce063da9c8b588a12,"['549d25df9a794775979561d2', '58abcf539a7947f1...",19038,1.142,38.355
620730cf9a7947ab96a44e27hk7puWJwSziw0m3sfTkKWA,Госсекретарь Швеции заявила о нежелании вступа...,2022-02-12 04:24:02,hk7puWJwSziw0m3sfTkKWA,[],0.000,5409f11ce063da9c8b588a12,"['5430f451cbb20f73931ecd05', '5409f15de063daa0...",51151,1.185,36.424


### Замена данных (Дата, session, authors,  category, tags, authors)

Создаем объединенный датафрейм трейна и теста. Заменим там перемнные и разобьем его обратно

In [5]:
# , "tags"
frames = [df_train[["category", "session", "authors"]], df_test[["category", "session", "authors"]]]
df_temp = pd.concat(frames)

Заменяем категориальные значения на int

In [6]:
df_temp["category"] = df_temp["category"].astype('category')
df_temp["category"] = df_temp["category"].cat.codes
df_temp["category"] = df_temp["category"].astype('int')

df_temp["authors"] = df_temp["authors"].astype('category')
df_temp["authors"] = df_temp["authors"].cat.codes
df_temp["authors"] = df_temp["authors"].astype('int')

df_temp["session"] = df_temp["session"].astype('category')
df_temp["session"] = df_temp["session"].cat.codes
df_temp["session"] = df_temp["session"].astype('int')

#df_temp["tags"] = df_temp["tags"].astype('category')
#df_temp["tags"] = df_temp["tags"].cat.codes
#df_temp["tags"] = df_temp["tags"].astype('int')

Мерджим полученные данные с траэйном и тестом

In [7]:

df_train = pd.merge(
    df_train,
    df_temp,
    how="left",
    left_index=True,
    right_index=True,
    sort=False,
    suffixes=("", "_code"),
    copy=True,
    indicator=False,
    validate=None,
)

df_test = pd.merge(
    df_test,
    df_temp,
    how="left",
    left_index=True,
    right_index=True,
    sort=False,
    suffixes=("", "_code"),
    copy=True,
    indicator=False,
    validate=None,
)

### Добавляем фичи

Добавим число тегов и авторов

In [8]:
for dataset in [df_train, df_test]:
    dataset['n_authors'] = dataset['authors'].map(lambda x: 0 if x.split(',') == ['[]'] else len(x.split(',')) )
    dataset['n_tags'] = dataset['tags'].map(lambda x: 0 if x.split(',') == ['[]'] else len(x.split(',')) )

Добавим день, меся, год

In [9]:
for dataset in [df_train, df_test]:
    dataset['day'] = pd.to_datetime(dataset['publish_date']).dt.strftime("%d").astype(int)
    dataset['mounth'] = pd.to_datetime(dataset['publish_date']).dt.strftime("%m").astype(int)
    dataset['year'] = pd.to_datetime(dataset['publish_date']).dt.strftime("%y").astype(int)

In [10]:
df_train.head(3)

,title,publish_date,session,authors,ctr,category,tags,views,depth,full_reads_percent,category_code,session_code,authors_code,n_authors,n_tags,day,mounth,year
document_id,,,,,,,,,,,,,,,,,,
624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ,Европейский банк развития приостановил доступ ...,2022-04-04 10:29:44,IDE7mtH4RBqGn-8MXfGffQ,[],1.580,5409f11ce063da9c8b588a18,"['55928d339a794751dc8303d6', '542d1e28cbb20f86...",20460,1.134,35.850,2,2405,739,0,4,4,4,22
620f6b899a7947701cf489e1KtVJsteHStO5oditt3Uvzw,Кремль назвал регулярным процессом учебные зап...,2022-02-18 10:00:39,KtVJsteHStO5oditt3Uvzw,"['54244e01cbb20f03076b236d','5878a2ec9a7947e53...",1.853,5409f11ce063da9c8b588a12,"['549d25df9a794775979561d2', '58abcf539a7947f1...",19038,1.142,38.355,0,2724,46,2,3,18,2,22
620730cf9a7947ab96a44e27hk7puWJwSziw0m3sfTkKWA,Госсекретарь Швеции заявила о нежелании вступа...,2022-02-12 04:24:02,hk7puWJwSziw0m3sfTkKWA,[],0.000,5409f11ce063da9c8b588a12,"['5430f451cbb20f73931ecd05', '5409f15de063daa0...",51151,1.185,36.424,0,5741,739,0,3,12,2,22


In [11]:
df_test.head(3)

,title,publish_date,session,authors,ctr,category,tags,category_code,session_code,authors_code,n_authors,n_tags,day,mounth,year
document_id,,,,,,,,,,,,,,,
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw,«Крайне провокационная тема»: как Россия и США...,2022-02-01 17:02:44,J0AvX96vTAaQCiWVbzoMdw,[],0.000,5409f11ce063da9c8b588a12,"['58ca77679a794700b1e37f0a', '5409f1b7e063daa0...",0,2510,739,0,5,1,2,22
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw,Власти Херсонской области пообещали сделать ру...,2022-05-24 00:50:55,QS5CqzXYRnmDdR2LaSreEw,[],1.598,5409f11ce063da9c8b588a12,"['5433603acbb20f6e5def0cc5', '5409f420e063daa0...",0,3427,739,0,4,24,5,22
627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw,Премьер и президент Финляндии призвали срочно ...,2022-05-12 07:11:51,XVsoyrUOT8OJJg2_finJhw,[],2.006,5409f11ce063da9c8b588a12,"['5409f270e063daa0f408b0a7', '5409f196e063daa0...",0,4312,739,0,3,12,5,22


In [12]:
df_train.n_authors.value_counts()

1     3457
0     2900
2      547
3       65
14      13
4       10
5        8
Name: n_authors, dtype: int64

## Выделим выборки

In [13]:
y = df_train[["views","depth","full_reads_percent"]]


X = df_train.drop(["views","depth","full_reads_percent", 
                   "title","publish_date", "session", "tags",
                   "authors", "category"], axis = 1)
test_df = df_test.drop(["title", "publish_date", "session", "tags", "authors", "category"], axis = 1)

In [14]:
X.head()

,ctr,category_code,session_code,authors_code,n_authors,n_tags,day,mounth,year
document_id,,,,,,,,,
624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ,1.580,2,2405,739,0,4,4,4,22
620f6b899a7947701cf489e1KtVJsteHStO5oditt3Uvzw,1.853,0,2724,46,2,3,18,2,22
620730cf9a7947ab96a44e27hk7puWJwSziw0m3sfTkKWA,0.000,0,5741,739,0,3,12,2,22
6262a5889a79470b78c9ca307UKY2SSZTjCcjhwBzxw37w,0.000,0,1049,739,0,5,22,4,22
626678929a79477ca0101568wuMYES90REuV5YhrN75IXg,0.000,5,7665,739,0,6,25,4,22


In [15]:
test_df.head()

,ctr,category_code,session_code,authors_code,n_authors,n_tags,day,mounth,year
document_id,,,,,,,,,
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw,0.000,0,2510,739,0,5,1,2,22
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw,1.598,0,3427,739,0,4,24,5,22
627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw,2.006,0,4312,739,0,3,12,5,22
628618629a7947d4927eb812upfii3whSSuMXCqcqF8VbQ,0.000,0,7405,739,0,4,19,5,22
620e76109a7947235623695b5hzCiIHdSYKQIr8WAM18bw,9.136,0,852,494,1,3,17,2,22


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Подбор модели

In [ ]:
%%time
#regr = RandomForestRegressor(random_state=0)
model_views = xg.XGBRegressor()
model_depth = xg.XGBRegressor()
model_full_reads_percent = xg.XGBRegressor()

Обучим модели

In [ ]:
%%time
model_views.fit(X_train, y_train['views'])
model_depth.fit(X_train, y_train['depth'])
model_full_reads_percent.fit(X_train, y_train['full_reads_percent'])

In [ ]:
X_train.columns

Предскажем значения

In [ ]:
pred_views = model_views.predict(X_test)
pred_depth = model_depth.predict(X_test)
pred_full_reads_percent = model_full_reads_percent.predict(X_test)

## Оценка точности на тесте

In [ ]:
score_views = r2_score(y_test["views"], pred_views)
print('score_views = ', score_views)

score_depth = r2_score(y_test["depth"], pred_depth)
print('score_depth = ', score_depth)

score_frp = r2_score(y_test["full_reads_percent"], pred_full_reads_percent)
print('score_frp = ', score_frp)

In [ ]:
score = 0.4 * score_views + 0.3 * score_depth + 0.3 * score_frp

print(score)

### Оценка точности на CV

Просто:

In [27]:
%%time
model_1 = xg.XGBRegressor()
scores_v = cross_validate(model_1, 
                          X, 
                          y['views'], cv=5, scoring='r2')
print('views :', scores_v['test_score'].mean())

model_1 = xg.XGBRegressor()
scores_d = cross_validate(model_1, 
                          X, 
                          y['depth'], cv=5, scoring='r2')
print('depth :', scores_d['test_score'].mean())

model_1 = xg.XGBRegressor()
scores_frp = cross_validate(model_1, 
                          X, 
                          y['full_reads_percent'], cv=5, scoring='r2')
print('full_reads_percent :', scores_frp['test_score'].mean())

print(0.4 * scores_v['test_score'].mean() + 0.3 * scores_d['test_score'].mean() + 0.3 * scores_frp['test_score'].mean())

views : 0.4994086596049425
depth : 0.7446408562221991
full_reads_percent : 0.2839795983016149
0.5083496001991212
Wall time: 10.9 s


С дропом мешающих перменных

In [28]:
%%time
model_1 = xg.XGBRegressor()
scores_v = cross_validate(model_1, 
                          X.drop(['n_tags'],axis = 1), 
                          y['views'], cv=5, scoring='r2')
print('views :', scores_v['test_score'].mean())

model_1 = xg.XGBRegressor()
scores_d = cross_validate(model_1, 
                          X.drop(['session_code'],axis = 1), 
                          y['depth'], cv=5, scoring='r2')
print('depth :', scores_d['test_score'].mean())

model_1 = xg.XGBRegressor()
scores_frp = cross_validate(model_1, 
                          X.drop(['session_code'],axis = 1), 
                          y['full_reads_percent'], cv=5, scoring='r2')
print('full_reads_percent :', scores_frp['test_score'].mean())

print(0.4 * scores_v['test_score'].mean() + 0.3 * scores_d['test_score'].mean() + 0.3 * scores_frp['test_score'].mean())

views : 0.600797813944905
depth : 0.7598758977060135
full_reads_percent : 0.29297282678640857
0.5561737429256886
Wall time: 10.1 s


С добавлением в трейн full_reads_percent предсказанных значений для views и depth

In [52]:
model_views = xg.XGBRegressor()
model_depth = xg.XGBRegressor()
model_full_reads_percent = xg.XGBRegressor()

model_views.fit(X.drop(['n_tags'] , axis=1), y['views'])
X_to_frd = X.copy()

scores_v = cross_validate(model_views, 
                          X.drop(['n_tags'],axis = 1), 
                          y['views'], cv=5, scoring='r2')
print('views :', scores_v['test_score'].mean())

#X_to_frd['pred_views'] = model_views.predict(X.drop(['n_tags'], axis=1))

model_depth.fit(X.drop(['session_code'], axis=1), y['depth'])
X_to_frd['pred_depth'] = model_depth.predict(X.drop(['session_code'], axis=1))

scores_d = cross_validate(model_depth, 
                          X.drop(['session_code'],axis = 1), 
                          y['depth'], cv=5, scoring='r2')
print('depth :', scores_d['test_score'].mean())

#model_full_reads_percent.fit(X_to_frd.drop(['session_code'], axis=1), y['full_reads_percent'])


scores_frp = cross_validate(model_full_reads_percent, 
                          X_to_frd.drop(['session_code'], axis=1), 
                          y['full_reads_percent'], cv=5, scoring='r2')
print('full_reads_percent :', scores_frp['test_score'].mean())

print(0.4 * scores_v['test_score'].mean() + 0.3 * scores_d['test_score'].mean() + 0.3 * scores_frp['test_score'].mean())

views : 0.600797813944905
depth : 0.7598758977060135
full_reads_percent : 0.31216970981932657
0.561932807835564


### Исследуем модель для views

In [24]:
'''
model_1 = xg.XGBRegressor()
scores_v = cross_validate(model_1, X, y['views'], cv=5, scoring='r2')
print('Используя все фичи: ', scores_v['test_score'].mean())

for fiature in ['category_code', 'year', 'day', 'n_tags', 'session_code']:
    model_1 = xg.XGBRegressor()
    scores_v = cross_validate(model_1, 
                              X.drop(fiature, axis = 1),
                              y['views'], cv=5, scoring='r2')
    print('Без ',fiature,':', scores_v['test_score'].mean())
'''

Используя все фичи:  0.4994086596049425
Без  category_code : 0.49595772685940903
Без  year : 0.49666517466100546
Без  day : 0.414306881850883
Без  n_tags : 0.600797813944905
Без  session_code : 0.4937568889859902
Wall time: 21.2 s


Вывод - можно попробовать убрать n_tags

In [ ]:
"""X_local = X.drop(['n_tags'],axis = 1)

model_1 = xg.XGBRegressor()
scores_v = cross_validate(model_1, X_local, y['views'], cv=5, scoring='r2')
print('Используя все фичи (без n_tags): ', scores_v['test_score'].mean())

for fiature in ['category_code', 'year', 'day', 'session_code']:
    model_1 = xg.XGBRegressor()
    scores_v = cross_validate(model_1, 
                              X_local.drop(fiature, axis = 1),
                              y['views'], cv=5, scoring='r2')
    print('Без ',fiature,':', scores_v['test_score'].mean())
"""

Вывод: Оставляем как есть

### Исследуем модель для depth

In [ ]:
"""model_1 = xg.XGBRegressor()
scores_v = cross_validate(model_1, X, y['depth'], cv=5, scoring='r2')
print('Используя все фичи: ', scores_v['test_score'].mean())

for fiature in ['ctr', 'category_code', 'authors_code','session_code', 'n_tags', 'day', 'year']:
    model_1 = xg.XGBRegressor()
    scores_v = cross_validate(model_1, 
                              X.drop(fiature, axis = 1),
                              y['depth'], cv=5, scoring='r2')
    print('Без ',fiature,':', scores_v['test_score'].mean())
"""

Вывод - Можно попробоавть без session_code

In [ ]:
'''
X_local = X.drop(['session_code'],axis = 1)

model_1 = xg.XGBRegressor()
scores_v = cross_validate(model_1, X_local, y['depth'], cv=5, scoring='r2')
print('Используя все фичи (без n_tags): ', scores_v['test_score'].mean())

for fiature in ['ctr', 'category_code', 'authors_code', 'n_tags', 'day', 'year']:
    model_1 = xg.XGBRegressor()
    scores_v = cross_validate(model_1, 
                              X_local.drop(fiature, axis = 1),
                              y['depth'], cv=5, scoring='r2')
    print('Без ',fiature,':', scores_v['test_score'].mean())
'''

Оставляем как есть

### Исследуем модель для full_reads_percent

In [ ]:
'''
model_1 = xg.XGBRegressor()
scores_v = cross_validate(model_1, X, y['full_reads_percent'], cv=5, scoring='r2')
print('Используя все фичи: ', scores_v['test_score'].mean())

for fiature in ['n_authors', 'year', 'day', 'authors_code', 'session_code']:
    model_1 = xg.XGBRegressor()
    scores_v = cross_validate(model_1, 
                              X.drop(fiature, axis = 1),
                              y['full_reads_percent'], cv=5, scoring='r2')
    print('Без ',fiature,':', scores_v['test_score'].mean())
'''

Вывод - Можно попробовать убрать Session_code

In [ ]:
'''
X_local = X.drop(['session_code'],axis = 1)

model_1 = xg.XGBRegressor()
scores_v = cross_validate(model_1, X_local, y['full_reads_percent'], cv=5, scoring='r2')
print('Используя все фичи (без n_tags): ', scores_v['test_score'].mean())

for fiature in ['n_authors', 'year', 'day', 'authors_code']:
    model_1 = xg.XGBRegressor()
    scores_v = cross_validate(model_1, 
                              X_local.drop(fiature, axis = 1),
                              y['full_reads_percent'], cv=5, scoring='r2')
    print('Без ',fiature,':', scores_v['test_score'].mean())
'''

Вывод: Оставляем как есть

### Итог: Для View - n_tags; Для двух других - Session_code

#### Фигачим ответ

In [37]:
X

,ctr,category_code,session_code,authors_code,n_authors,n_tags,day,mounth,year
document_id,,,,,,,,,
624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ,1.580,2,2405,739,0,4,4,4,22
620f6b899a7947701cf489e1KtVJsteHStO5oditt3Uvzw,1.853,0,2724,46,2,3,18,2,22
620730cf9a7947ab96a44e27hk7puWJwSziw0m3sfTkKWA,0.000,0,5741,739,0,3,12,2,22
6262a5889a79470b78c9ca307UKY2SSZTjCcjhwBzxw37w,0.000,0,1049,739,0,5,22,4,22
626678929a79477ca0101568wuMYES90REuV5YhrN75IXg,0.000,5,7665,739,0,6,25,4,22
...,...,...,...,...,...,...,...,...,...
61fd479c9a7947848183694751RC_1i5QlGuv7B2FzwTjw,2.354,5,763,173,1,8,4,2,22
628a8f1c9a79478c64a7dae450tIkSGVSf2O8XQgk_uaHQ,4.112,3,762,569,2,3,22,5,22
6244cf5e9a79479099346b26tKlqoxQYSmij0ZBrGzebug,5.125,5,7228,739,0,4,30,3,22


In [48]:
test_df

,ctr,category_code,session_code,authors_code,n_authors,n_tags,day,mounth,year
document_id,,,,,,,,,
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw,0.000,0,2510,739,0,5,1,2,22
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw,1.598,0,3427,739,0,4,24,5,22
627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw,2.006,0,4312,739,0,3,12,5,22
628618629a7947d4927eb812upfii3whSSuMXCqcqF8VbQ,0.000,0,7405,739,0,4,19,5,22
620e76109a7947235623695b5hzCiIHdSYKQIr8WAM18bw,9.136,0,852,494,1,3,17,2,22
...,...,...,...,...,...,...,...,...,...
623ba6bc9a79470214cb9418x4OZQ1IjTb27nPAgv5wAZg,1.788,4,7695,358,1,4,23,3,22
624fda1f9a79471a9bd102eetuRTrLPGSZyNbc9_YcBAeQ,4.427,3,7300,417,1,2,8,4,22
6257a6a99a79477eea75b86dtBr3UG9ORW-gOybkVKX6ww,0.000,1,7214,739,0,3,14,4,22


In [39]:
%%time
model_views = xg.XGBRegressor()
model_depth = xg.XGBRegressor()
model_full_reads_percent = xg.XGBRegressor()

Wall time: 0 ns


In [40]:
%%time
model_views.fit(X.drop(['n_tags'] , axis=1), y['views'])
model_depth.fit(X.drop(['session_code'], axis=1), y['depth'])

#model_full_reads_percent.fit(X.drop(['session_code'], axis=1), y['full_reads_percent'])

Wall time: 1.59 s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [45]:
X_to_frp = X.copy()
X_to_frp['pred_depth'] = model_depth.predict(X.drop(['session_code'], axis=1))

In [46]:
model_full_reads_percent.fit(X_to_frp.drop(['session_code'], axis=1), y['full_reads_percent'])

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [49]:
sample['views'] = model_views.predict(test_df.drop(['n_tags'], axis=1))
sample['depth'] = model_depth.predict(test_df.drop(['session_code'], axis=1))

test_df_to_frp = test_df.copy()
test_df_to_frp['pred_depth'] = sample['depth']

sample['full_reads_percent'] = model_full_reads_percent.predict(test_df_to_frp.drop(['session_code'], axis=1))


In [50]:
sample

,views,depth,full_reads_percent
document_id,,,
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw,20325.433594,1.172853,39.752182
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw,9841.697266,1.066869,36.287079
627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw,9595.762695,1.070459,40.037220
628618629a7947d4927eb812upfii3whSSuMXCqcqF8VbQ,7280.983398,1.066451,41.413002
620e76109a7947235623695b5hzCiIHdSYKQIr8WAM18bw,103764.703125,1.135663,30.940859
...,...,...,...
623ba6bc9a79470214cb9418x4OZQ1IjTb27nPAgv5wAZg,47986.593750,1.134984,33.729546
624fda1f9a79471a9bd102eetuRTrLPGSZyNbc9_YcBAeQ,27891.960938,1.126167,32.611313
6257a6a99a79477eea75b86dtBr3UG9ORW-gOybkVKX6ww,5098.840332,1.062279,42.337460


In [51]:
sample.to_csv('content/predictions/test_prediction_v4.csv')

#### Public Score: 0.648662 - стало хуже

### На кросс валидации скор вырос до 0.5619; По идее оно работает, но не попадает в тест